In [1]:
import math
import os
import random
import shutil
import PIL
import pandas as pd
import numpy as np
from pdf2image import convert_from_path
import matplotlib.pyplot as plt
import string
import cv2
import tensorflow as tf
from PIL.Image import Image
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
#from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers import Concatenate
from keras.layers.core import Lambda, Flatten, Dense, Dropout
from keras.preprocessing import image
#from keras.utils import to_categorical
from keras import initializers
#from keras.engine.topology import Layer
from tensorflow.keras.optimizers import Adam
from keras import backend as K
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
#import SciPy
import keras
from tensorflow.keras import metrics
from tensorflow.keras.applications.resnet50 import ResNet50
from sklearn.metrics import confusion_matrix
import shutil
def writer_to_folder(path,newpath):
    dic={}
    for filename in  os.listdir(path):
        writer=filename.split('_')[0]
        if writer in dic:
            dic[writer].append(filename)
        else:
            dic[writer]=[]
            dic[writer].append(filename)

    
    for keyed in dic.items():
        os.makedirs(newpath+'/'+keyed[0])
        for i in keyed[1]:
            try:
                shutil.copy(path+'/'+i, newpath+'/'+keyed[0]+'/')
                print("File copied successfully.")
            except shutil.SameFileError:
                 print("Source and destination represents the same file.")
 
        
    # data = pd.DataFrame(columns=["image", "writer"])

def loadImages(path = "./Dataset"):
    """
    returns a dataframe that saves image name and writer in each row
    """
    data = pd.DataFrame(columns=["image", "writer"])

    for writer in os.listdir(path):
        writer_path = os.path.join(path,writer)

        for image in os.listdir(os.path.join(path,writer)):
            data.loc[len(data)] = [image, writer]

        data.to_csv('Writerset.csv', index=False)

    return data

loadImages("./DatasetTrainWriters")




def base_network(input_dim):
    inputs = Input(shape=input_dim)
    conv_1=Conv2D(64,(5,5),padding="same",activation='relu',name='conv_1')(inputs)
    conv_1=MaxPooling2D(pool_size=(2, 2))(conv_1)
    conv_2=Conv2D(128,(5,5),padding="same",activation='relu',name='conv_2')(conv_1)
    conv_2=MaxPooling2D(pool_size=(2, 2))(conv_2)
    conv_3=Conv2D(256,(3,3),padding="same",activation='relu',name='conv_3')(conv_2)
    conv_3=MaxPooling2D(pool_size=(2, 2))(conv_3)
    conv_4=Conv2D(512,(3,3),padding="same",activation='relu',name='conv_4')(conv_3)
    conv_5=Conv2D(512,(3,3),padding="same",activation='relu',name='conv_5')(conv_4)
    conv_5=MaxPooling2D(pool_size=(2, 2))(conv_5)

    dense_1=Flatten()(conv_5)
    dense_1=Dense(512,activation="relu")(dense_1)
    dense_1=Dropout(0.3)(dense_1)
    dense_2=Dense(512,activation="relu")(dense_1)
    dense_2=Dropout(0.5)(dense_2)
    return Model(inputs, dense_2)

input_shape=(64,64,1)
learning_rate=0.000001

base_network = base_network(input_shape)
input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)
processed_a = base_network(input_a)
processed_b = base_network(input_b)

fc6=concatenate([processed_a, processed_b])

fc7 = Dense(4096, activation = 'relu')(fc6)
fc7 = Dropout(0.7)(fc7)
fc8 = Dense(4096, activation = 'relu')(fc7)
fc8 = Dropout(0.8)(fc8)

fc9=Dense(1, activation='sigmoid')(fc8)
model = Model([input_a, input_b], fc9)

def splitData(path = "Writerset.csv"):
    """
    splits data to train and test
    returns test writers for page level functions
    """
    data = pd.read_csv(path)

    writers = set(data["writer"])
    writers = list(writers)
    random.shuffle(writers)
    writers_len = len(list(writers))
    train = writers[:round(writers_len * 0.80)]
    test = writers[round(writers_len * 0.80):]

    train_df = data.loc[data['writer'].isin(train)]
    test_df = data.loc[data['writer'].isin(test)]

    return train_df, test_df, test

def loadImageDict(df, path = "./aug_patches", khatt = 0):
    """
    :param df: df which we will load images from (columns: image, writer)
    :param path: path to load images from
    :return: a dict with image names as keys and loaded images as values
    """
    dict = {}
    for image in df['image'].values:
        if khatt == 0: # regular data
            index = image.find("_", image.find("_")+1) # find index of second _ in order to find folder name(= original image name)
            folder = image[:index]  # original image name
        if khatt == 1:
            if (len(image)==24):
             folder = image[:len(image) - 14] #original image name
            else:
             folder = image[:len(image) - 12] #original image name

            print(folder)
        folder_path = os.path.join(path,folder)

        # load image
        loaded = cv2.resize(cv2.imread(folder_path + "/" + image, cv2.IMREAD_GRAYSCALE), (64, 64))
        thresh, loaded = cv2.threshold(loaded, 180, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        loaded = np.array(loaded, dtype=np.float32)
        #loaded/=255

        # add to dictionary
        dict[image] = loaded

    return dict

def newcreatePairs(df, dict, pairs_num):
    """
    :param df: df (with colums image, writer) to create pairs from
    :param dict: a dict with image names as keys and loaded images as values (from loadImageDict)
    :param pairs_num: pairs_num of pairs for same writer and pairs_num for different writer
    :return: 3 numpy arrays of left, right and labels
    """
    left = []
    right = []
    labels = []

    for index, row in df.iterrows():
        image = row["image"]
        writer = row["writer"]

        writer_images = df.loc[df['writer'] == writer]
        writer_images = writer_images["image"].values
        diff_writer_images = df.loc[df['writer'] != writer]
        diff_writer_images = diff_writer_images["image"].values

        # add pairs for same writer
        filtered_images = np.delete(writer_images, np.where(writer_images == image))
        try:
            right_patches = np.random.choice(filtered_images, size=pairs_num, replace=False)
        except:
            right_patches = np.random.choice(filtered_images, size=len(filtered_images), replace=False)
        for p in right_patches:
            left.append(dict[image])
            right.append(dict[p])
            labels.append(1)

        # add pairs for different writer
        try:
            right_patches = np.random.choice(diff_writer_images, size=pairs_num, replace=False)
        except:
            right_patches = np.random.choice(diff_writer_images, size=len(diff_writer_images), replace=False)
        for p in right_patches:
            left.append(dict[image])
            right.append(dict[p])
            labels.append(0)

        # shuffle
        # temp = list(zip(left, right, labels))
        # random.shuffle(temp)
        # left, right, labels = zip(*temp)
        # left, right, labels = list(left), list(right), list(labels)

    return np.array(left), np.array(right), np.array(labels).astype("float32")


 #training
adam=Adam(learning_rate=learning_rate)
loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)


    #adam = Adam(learning_rate=learning_rate)
    #model = siamese_model((150, 150, 1))

train_df, test_df, _ = splitData("patches_aug.csv")
train_df = train_df.sample(frac = 1)
test_df = test_df.sample(frac = 1)

train_writers = train_df['writer'].unique()
test_writers = test_df['writer'].unique()

dic = loadImageDict(train_df, "./aug_patches", khatt = 1)
test_dic = loadImageDict(test_df, "./aug_patches", khatt = 1)
print("finished building dictionaries")

img_list = list(dic.values())
print(img_list[0].shape)


left, right, labels = newcreatePairs(train_df, dic, 1)
test_left, test_right, test_labels = newcreatePairs(test_df, test_dic, 1)

print("finished create pairs")


2022-12-21 03:43:45.196385: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-21 03:43:50.968043: I tensorflow/compiler/xla/stream_executor/rocm/rocm_gpu_executor.cc:843] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-21 03:43:51.942409: I tensorflow/compiler/xla/stream_executor/rocm/rocm_gpu_executor.cc:843] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-21 03:43:51.942727: I tensorflow/compiler/xla/stream_executor/rocm/rocm_gpu_executor.cc:843] successful NUMA node read from SysFS had negative value (-1), but th

AHTD3A0869
AHTD3A0757
AHTD3A1046
AHTD3A0797
AHTD3A0655
AHTD3A0873
AHTD3A0757
AHTD3A0995
AHTD3A0723
AHTD3A0637
AHTD3A0933
AHTD3A0613
AHTD3A0215
AHTD3A0707
AHTD3A0612
AHTD3A0682
AHTD3A0310
AHTD3A1011
AHTD3A0104
AHTD3A0612
AHTD3A0713
AHTD3A0368
AHTD3A0873
AHTD3A0604
AHTD3A0553
AHTD3A0792
AHTD3A0699
AHTD3A0918
AHTD3A0497
AHTD3A0847
AHTD3A0439
AHTD3A0842
AHTD3A0714
AHTD3A0440
AHTD3A0818
AHTD3A0669
AHTD3A1086
AHTD3A0836
AHTD3A0694
AHTD3A0424
AHTD3A1001
AHTD3A0155
AHTD3A0997
AHTD3A1144
AHTD3A0624
AHTD3A0694
AHTD3A0292
AHTD3A0295
AHTD3A0685
AHTD3A0747
AHTD3A0912
AHTD3A0789
AHTD3A0757
AHTD3A0578
AHTD3A0872
AHTD3A0475
AHTD3A0853
AHTD3A0835
AHTD3A0375
AHTD3A0373
AHTD3A0535
AHTD3A0650
AHTD3A0273
AHTD3A0406
AHTD3A0284
AHTD3A0856
AHTD3A0875
AHTD3A0576
AHTD3A1059
AHTD3A0580
AHTD3A0823
AHTD3A0585
AHTD3A1018
AHTD3A0584
AHTD3A1026
AHTD3A0539
AHTD3A0663
AHTD3A0251
AHTD3A1075
AHTD3A0837
AHTD3A0755
AHTD3A0641
AHTD3A0160
AHTD3A0666
AHTD3A0737
AHTD3A1020
AHTD3A0693
AHTD3A0842
AHTD3A0667
AHTD3A0304
AHTD3A0682

In [2]:
import tensorflow.compat.v1 as tff
tff.compat.v1.disable_eager_execution()

In [31]:
FLAGS = None

def deepnn(x, train):
  with tff.name_scope('reshape'):
    x_image = tff.reshape(x, [-1, 8192])
  with tff.name_scope('fc0'):
    W_fc0 = weight_variable([8192, 4096])
    b_fc0 = bias_variable([4096]) 
    h_fc0 = tff.nn.relu(tff.matmul(x_image, W_fc0) + b_fc0)
  with tff.name_scope('fc0.5'):
    W_fc05 = weight_variable([4096, 2048])
    b_fc05= bias_variable([2048]) 
    h_fc05 = tff.nn.relu(tff.matmul(h_fc0, W_fc05) + b_fc05)
  with tff.name_scope('fc1'):
    W_fc1 = weight_variable([2048, 1024])
    b_fc1 = bias_variable([1024]) 
    h_fc1 = tff.nn.relu(tff.matmul(h_fc05, W_fc1) + b_fc1)
    
  with tff.name_scope('fc2'):
    W_fc2 = weight_variable([1024, 512])
    b_fc2 = bias_variable([512]) 
    h_fc2 = tff.nn.relu(tff.matmul(h_fc1, W_fc2) + b_fc2)

  with tff.name_scope('fc3'):
    W_fc3 = weight_variable([512, 128])
    b_fc3 = bias_variable([128]) 
    h_fc3 = tff.nn.relu(tff.matmul(h_fc2, W_fc3) + b_fc3)    
    
  with tff.name_scope('fc4'):
    W_fc4 = weight_variable([128, 1])
    b_fc4 = bias_variable([1])
    y_conv = tff.sigmoid(tff.matmul(h_fc3, W_fc4) + b_fc4)
    return y_conv, h_fc3

def weight_variable(shape):
  """weight_variable generates a weight variable of a given shape."""
  initial = tff.truncated_normal(shape, stddev=0.05)
  return tff.Variable(initial)

def bias_variable(shape):
  """bias_variable generates a bias variable of a given shape."""
  initial = tff.constant(0.1, shape=shape)
  return tff.Variable(initial)

def get_batch(name_pos, label_pos, name_neg, label_neg, batch_size_p, batch_size_n, now_batch_p, now_batch_n, total_batch_n):
    image_batch_pos = name_pos[now_batch_p*batch_size_p:(now_batch_p+1)*batch_size_p,:]
    label_batch_pos = label_pos[now_batch_p*batch_size_p:(now_batch_p+1)*batch_size_p]
    image_batch_neg = name_neg[now_batch_n*batch_size_n:(now_batch_n+1)*batch_size_n,:]
    label_batch_neg = label_neg[now_batch_n*batch_size_n:(now_batch_n+1)*batch_size_n]        
    batchdata=np.concatenate((image_batch_pos,image_batch_neg),axis=0)        
    batchlabel=np.concatenate((label_batch_pos,label_batch_neg),axis=0)
    return batchdata, batchlabel

def loss_with_top_rank(logits, labels, p):
    
  with tff.name_scope('toprank'):
      
      loss = tff.zeros([1,1])
            
      num = tff.constant(1/5)
            
      index1 = tff.where(tff.equal(labels,1))
            
      indices1 = index1[:,0]
            
      index2 = tff.where(tff.not_equal(labels,1))
            
      indices2 = index2[:,0] 
            
      cs=tff.constant(1.0)
        
      p1 = p
        
      p2 = cs/p1
            
      sum_n = tff.zeros([1,1])
            
      sum_p = tff.zeros([1,1])
            
      l = tff.zeros([1,1])
            
      feature = tff.zeros([1,1])
            
      norm_feature = tff.zeros([1,1])
            
      l_p = tff.zeros([1,1])
            
      for u in range(5):
            
          for v in range(40):
                  
              feature = tff.subtract(logits[indices1[u]],logits[indices2[v]])
                
              norm_feature =tff.log(1+tff.exp(-feature))
                
              l_p = tff.pow(norm_feature,p1)
                
              sum_n = sum_n+l_p
    
          l = tff.pow(sum_n,p2)
              
          sum_p = sum_p+l
              
      loss = tff.multiply(sum_p,num)         
          
      return loss

array([[  0., 255.,   0., ...,   0.,   0.,   0.],
       [  0., 255.,   0., ...,   0.,   0.,   0.],
       [  0., 255.,   0., ..., 255., 255.,   0.],
       ...,
       [255.,   0., 255., ...,   0., 255.,   0.],
       [  0.,   0.,   0., ..., 255.,   0.,   0.],
       [255.,   0.,   0., ..., 255.,   0.,   0.]], dtype=float32)

In [5]:
def gettrue(left,right,labels,lb):
    llsstt=[]
    for i in range(len(left)):
        if labels[i]==lb:
            llsstt.append(i)
    arr1=np.take(left, llsstt, 0)
    arr2=np.take(right, llsstt, 0)
    arrr=[]
    
    for i in range(len(arr1)):
        arrr.append([arr1[i],arr2[i]])
    return np.array(arrr)
    
stff=gettrue(left,right,labels,0)
print(stff)
        

[[[[  0.   0.   0. ...   0.   0.   0.]
   [  0.   0.   0. ...   0.   0.   0.]
   [  0.   0.   0. ...   0.   0.   0.]
   ...
   [  0.   0.   0. ...   0.   0.   0.]
   [  0.   0.   0. ...   0.   0.   0.]
   [  0.   0.   0. ...   0.   0.   0.]]

  [[  0.   0.   0. ...   0.   0.   0.]
   [  0.   0.   0. ...   0.   0.   0.]
   [  0.   0.   0. ...   0.   0.   0.]
   ...
   [  0.   0.   0. ...   0.   0.   0.]
   [  0.   0.   0. ...   0.   0. 255.]
   [  0.   0.   0. ...   0.   0.   0.]]]


 [[[  0.   0.   0. ...   0.   0.   0.]
   [  0.   0.   0. ...   0.   0.   0.]
   [  0.   0.   0. ...   0.   0.   0.]
   ...
   [255. 255. 255. ...   0.   0. 255.]
   [255. 255.   0. ...   0.   0.   0.]
   [255. 255.   0. ...   0.   0.   0.]]

  [[  0.   0.   0. ...   0.   0.   0.]
   [  0.   0.   0. ...   0.   0.   0.]
   [  0.   0.   0. ...   0.   0.   0.]
   ...
   [  0.   0.   0. ...   0.   0.   0.]
   [  0.   0.   0. ...   0.   0.   0.]
   [  0.   0.   0. ...   0.   0.   0.]]]


 [[[  0.   0.   0. ...  

In [33]:
for fold in range(5):
        p_array=[2,4,8,16,32]
        num_p=p_array[fold]
        p_value=np.float32(num_p)
        p_value=np.reshape(p_value,[1])
        tff.reset_default_graph()
        model_name = 'doc_id'+'_p_' + str(num_p)
        training = tff.placeholder(tff.bool)    
        x = tff.placeholder(tff.float32, [None, 8192])
        y_ = tff.placeholder(tff.int64, [None,1])
        y_conv,fc1 = deepnn(x, training)

        with tff.name_scope('loss'):
    
            toprankloss = loss_with_top_rank(logits=y_conv, labels=y_, p=p_value)
    
            toprankloss = tff.reshape(toprankloss,[])

            initial_loss= tff.constant(1e-14)

            toprankloss = tff.add(toprankloss,initial_loss)

            loss_summary = tff.summary.scalar('loss', toprankloss)         
  

        global_step = tff.Variable(0, trainable=False)
        initial_learning_rate = 0.0001
        learning_rate = tff.train.exponential_decay(initial_learning_rate,
                                           global_step,
                                           decay_steps=100,decay_rate=0.90)
        with tff.name_scope('adam_optimizer'):
            update_ops = tff.get_collection(tff.GraphKeys.UPDATE_OPS)
        with tff.control_dependencies(update_ops):
            train_step = tff.train.AdamOptimizer(learning_rate).minimize(toprankloss)
        add_global = global_step.assign_add(1)

        sess=tff.Session()
        train_writer = tff.summary.FileWriter('./summary' + str(num_p)+ '/'+ 'p_' + str(num_p)+'doc_id', sess.graph)
        sess.run(tff.global_variables_initializer())
        saver = tff.train.Saver()                  
        step=0
        for epoch in range(10):
            # file=joblib.load('/home/xiaotong/Desktop/Experiments/Codes/Signature/dataset/BHSig260/'+di.dataset+'/resized/')
            # train=file['train_pairs']
            # traindata_pos,traindata_neg=train['G-G'],train['G-F']
            traindata_pos=gettrue(left,right,labels,1)
            traindata_neg=gettrue(left,right,labels,0)
            train_g_len=len(traindata_pos)
            train_f_len=len(traindata_neg)
            trainlabel_p=np.ones([train_g_len])
            trainlabel_n=np.zeros([train_f_len])
            now_batch_p=0
            now_batch_n=0
            batch_size_p=5
            batch_size_n=40
            total_batch_n=np.int(train_f_len/batch_size_n)+1
            num_of_pos=np.int(train_g_len/batch_size_p)
            print(len(traindata_pos))
            num_of_neg=np.int(train_f_len/batch_size_n)
            train_p=traindata_pos
            train_n=traindata_neg
            print(num_of_pos)
            rrrlo=np.int(num_of_pos/20)
            for i in range(rrrlo):
                now_batch_p=i
                if i < num_of_neg:
                    x_batch_train,y_batch_train=get_batch(train_p,trainlabel_p,train_n,trainlabel_n,batch_size_p,batch_size_n,now_batch_p,now_batch_n,total_batch_n)
                if i >= num_of_neg:
                    if now_batch_n % num_of_neg == 0:
                        now_batch_n = 0
                        train_n = traindata_neg
                    x_batch_train,y_batch_train = get_batch(train_p,trainlabel_p,train_n,trainlabel_n,batch_size_p,batch_size_n,now_batch_p,now_batch_n,total_batch_n)
                x_batch_train=np.float32(x_batch_train)
                x_batch_train=np.reshape(x_batch_train,[-1, 8192])
                y_batch_train=np.reshape(y_batch_train,[45,1])
                g, rate = sess.run([add_global, learning_rate])
                print(x_batch_train.shape,' *** ', y_batch_train.shape)
                loss,losssummary, _ = sess.run([toprankloss,loss_summary,train_step], feed_dict={x: x_batch_train, y_ : y_batch_train, training:True})
                train_writer.add_summary(losssummary, step)
                # train_writer.add_summary(sess.graph)
                print("step: {}, train loss: {:g}".format(step, loss))
                print("i: ",i," of ",num_of_pos)
                now_batch_n=now_batch_n+1                
                step=step+1
            if (epoch % 1 == 0):
                print("Saving checkpoint")
                saver.save(sess, './ck'+'/'+str(num_p)+'/' + model_name + '_fc.ckpt')

2022-12-21 04:02:07.516512: I tensorflow/compiler/xla/stream_executor/rocm/rocm_gpu_executor.cc:843] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-21 04:02:07.516768: I tensorflow/compiler/xla/stream_executor/rocm/rocm_gpu_executor.cc:843] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-21 04:02:07.516819: I tensorflow/compiler/xla/stream_executor/rocm/rocm_gpu_executor.cc:843] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-21 04:02:07.516883: I tensorflow/compiler/xla/stream_executor/rocm/rocm_gpu_executor.cc:843] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-21 04:02:07.516963: I tensorflow/compiler/xla/stream_executo

22400
4480


2022-12-21 04:02:09.938323: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.


(45, 8192)  ***  (45, 1)


2022-12-21 04:02:12.267448: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.


...

(45, 8192)  ***  (45, 1)


2022-12-21 04:09:40.690151: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.


step: 0, train loss: 0.800957
i:  0  of  4480
(45, 8192)  ***  (45, 1)
step: 1, train loss: 0.801591
i:  1  of  4480
(45, 8192)  ***  (45, 1)
step: 2, train loss: 0.802866
i:  2  of  4480
(45, 8192)  ***  (45, 1)
step: 3, train loss: 0.801591
i:  3  of  4480
(45, 8192)  ***  (45, 1)
step: 4, train loss: 0.801591
i:  4  of  4480
(45, 8192)  ***  (45, 1)
step: 5, train loss: 0.801591
i:  5  of  4480
(45, 8192)  ***  (45, 1)
step: 6, train loss: 0.801591
i:  6  of  4480
(45, 8192)  ***  (45, 1)
step: 7, train loss: 0.801591
i:  7  of  4480
(45, 8192)  ***  (45, 1)
step: 8, train loss: 0.801591
i:  8  of  4480
(45, 8192)  ***  (45, 1)
step: 9, train loss: 0.801591
i:  9  of  4480
(45, 8192)  ***  (45, 1)
step: 10, train loss: 0.801591
i:  10  of  4480
(45, 8192)  ***  (45, 1)
step: 11, train loss: 0.801591
i:  11  of  4480
(45, 8192)  ***  (45, 1)
step: 12, train loss: 0.801591
i:  12  of  4480
(45, 8192)  ***  (45, 1)
step: 13, train loss: 0.801591
i:  13  of  4480
(45, 8192)  ***  (45, 1

2022-12-21 04:09:45.836234: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.


22400
4480
(45, 8192)  ***  (45, 1)
step: 224, train loss: 0.801591
i:  0  of  4480
(45, 8192)  ***  (45, 1)
step: 225, train loss: 0.801591
i:  1  of  4480
(45, 8192)  ***  (45, 1)
step: 226, train loss: 0.800957
i:  2  of  4480
(45, 8192)  ***  (45, 1)
step: 227, train loss: 0.801591
i:  3  of  4480
(45, 8192)  ***  (45, 1)
step: 228, train loss: 0.801591
i:  4  of  4480
(45, 8192)  ***  (45, 1)
step: 229, train loss: 0.801591
i:  5  of  4480
(45, 8192)  ***  (45, 1)
step: 230, train loss: 0.801591
i:  6  of  4480
(45, 8192)  ***  (45, 1)
step: 231, train loss: 0.801591
i:  7  of  4480
(45, 8192)  ***  (45, 1)
step: 232, train loss: 0.801591
i:  8  of  4480
(45, 8192)  ***  (45, 1)
step: 233, train loss: 0.801591
i:  9  of  4480
(45, 8192)  ***  (45, 1)
step: 234, train loss: 0.801591
i:  10  of  4480
(45, 8192)  ***  (45, 1)
step: 235, train loss: 0.801591
i:  11  of  4480
(45, 8192)  ***  (45, 1)
step: 236, train loss: 0.801591
i:  12  of  4480
(45, 8192)  ***  (45, 1)
step: 237, t

In [34]:
y_conv

<tf.Tensor 'fc4/Sigmoid:0' shape=(None, 1) dtype=float32>

In [35]:
test_left.shape

(11200, 64, 64)

In [36]:
arrr_test=[]
for i in range(len(test_left)):
        arrr_test.append([test_left[i],test_right[i]])
arrr_test=np.array(arrr_test)
arrr_test=np.reshape(arrr_test,[-1, 8192])

In [37]:
arrr_test.shape

(11200, 8192)

In [38]:
kk=sess.graph.get_tensor_by_name("fc4/Sigmoid:0")
init=tff.global_variables_initializer()
sess.run(init)


2022-12-21 04:11:37.619198: W tensorflow/c/c_api.cc:291] Operation '{name:'fc4/Variable_1/Adam_1/Assign' id:16406 op device:{requested: '', assigned: ''} def:{{{node fc4/Variable_1/Adam_1/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](fc4/Variable_1/Adam_1, fc4/Variable_1/Adam_1/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2022-12-21 04:11:38.093896: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.


In [39]:
label_pred=sess.run(y_conv,feed_dict={x:arrr_test})

2022-12-21 04:11:40.241268: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.


In [40]:
label_pred.shape

(11200, 1)

In [51]:
label_pred

array([[4.072539e-37],
       [0.000000e+00],
       [0.000000e+00],
       ...,
       [0.000000e+00],
       [0.000000e+00],
       [0.000000e+00]], dtype=float32)

In [72]:
reallst=[]
for i in label_pred:
    reallst.append(round(i[0]))

In [73]:
reallst

[0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [74]:
reallst=np.array(reallst)

In [75]:
from sklearn.metrics import accuracy_score


In [76]:
accuracy_score(test_labels,reallst)

0.5014285714285714

In [71]:
lssstt

[4.072539e-37,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.99999905,
 0.0,
 9.7404304e-26,
 0.0,
 0.0,
 0.0,
 0.0,
 0.99999964,
 6.8419046e-28,
 1.9040456e-32,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 9.513985e-28,
 0.0,
 0.0,
 0.0,
 0.0,
 4.982281e-16,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 8.2429e-11,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2.0210338e-38,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 4.0227593e-37,
 5.470264e-35,
 0.0,
 1.0,
 3.242395e-36,
 2.4336787e-12,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 4.54225e-14,
 0.0,
 0.0,
 0.0,
 2.3212134e-31,
 0.0,
 2.5228752e-16,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.6907281e-20,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 9.686065e-07,
 1.0,
 0.0,
 0.0,
 0.0,
 3.53601e-18,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2.1754949e-11,
 0.0,
 0.0,
 0.0,

In [53]:
lssstt

0.045097083

In [42]:
test_labels

array([1., 0., 1., ..., 0., 1., 0.], dtype=float32)

In [52]:
sess=tff.Session()
sess.run(y_conv,feed_dict={x:arrr_test})

2022-12-21 02:16:06.770215: I tensorflow/compiler/xla/stream_executor/rocm/rocm_gpu_executor.cc:843] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-21 02:16:06.770456: I tensorflow/compiler/xla/stream_executor/rocm/rocm_gpu_executor.cc:843] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-21 02:16:06.770512: I tensorflow/compiler/xla/stream_executor/rocm/rocm_gpu_executor.cc:843] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-21 02:16:06.770580: I tensorflow/compiler/xla/stream_executor/rocm/rocm_gpu_executor.cc:843] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-21 02:16:06.770632: I tensorflow/compiler/xla/stream_executo

FailedPreconditionError: Graph execution error:

Detected at node 'fc1/MatMul/ReadVariableOp' defined at (most recent call last):
    File "/home/mo/anaconda3/envs/python39-env/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/mo/anaconda3/envs/python39-env/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/mo/anaconda3/envs/python39-env/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/home/mo/anaconda3/envs/python39-env/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/home/mo/anaconda3/envs/python39-env/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/home/mo/anaconda3/envs/python39-env/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asy

In [34]:
cpp=tff.keras.callbacks.ModelCheckpoint(filepath="./ck/32/doc_id_p_32_fcc.ckpt")

In [36]:
llll=cpp.model

AttributeError: 'NoneType' object has no attribute 'predict'

In [27]:
tff.keras.models.load_model("./ck/32/doc_id_p_32_fcc.ckpt")

OSError: Unable to open file (file signature not found)